In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch
from torchtext.legacy import data 
from torch.jit import script, trace
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import unicodedata
import codecs
import csv
import time
import random
import re
import os
import pickle
from io import open
import itertools
import math
import pandas as pd
from torchtext.legacy.data import Field, BucketIterator, LabelField, TabularDataset

BASE_DIR='/content/drive/MyDrive/seq2py'
#https://stackoverflow.com/questions/62922640/calling-a-function-in-a-different-python-file-using-google-colab
%cd $BASE_DIR/utils
import preprocess, helper,transformer
from transformer import Encoder,Decoder,Seq2Seq

/content/drive/MyDrive/seq2py/utils


In [ ]:
#Then set a random seed for deterministic results/reproducability.
SEED = 2434
random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

USE_CUDA = torch.cuda.is_available()
device = torch.device("cuda" if USE_CUDA else "cpu")
n_gpu = torch.cuda.device_count()
torch.cuda.get_device_name(0)

'Tesla T4'

In [ ]:
with open(os.path.join(BASE_DIR ,"model/model1_src_stio.pkl"),"rb") as f:
  src_stoi = pickle.load(f)
with open(os.path.join(BASE_DIR ,"model/model1_src_itos.pkl"),"rb") as f:
  src_itos = pickle.load(f)


with open(os.path.join(BASE_DIR ,"model/model1_trg_stio.pkl"),"rb") as f:
  trg_stoi = pickle.load(f)
with open(os.path.join(BASE_DIR ,"model/model1_trg_itos.pkl"),"rb") as f:
  trg_itos = pickle.load(f)

In [ ]:
INPUT_DIM = len(src_stoi)
OUTPUT_DIM = len(trg_stoi)
HID_DIM = 256
ENC_LAYERS = 3
DEC_LAYERS = 4
ENC_HEADS = 8
DEC_HEADS = 8
ENC_PF_DIM = 512
DEC_PF_DIM = 512
ENC_DROPOUT = 0.15
DEC_DROPOUT = 0.15

enc = Encoder(INPUT_DIM, 
              HID_DIM, 
              ENC_LAYERS, 
              ENC_HEADS, 
              ENC_PF_DIM, 
              ENC_DROPOUT, 
              device)


dec = Decoder(OUTPUT_DIM, 
              HID_DIM, 
              DEC_LAYERS, 
              DEC_HEADS, 
              DEC_PF_DIM, 
              DEC_DROPOUT,
              device)

In [ ]:
SRC_PAD_IDX = src_stoi['<pad>']
TRG_PAD_IDX = trg_stoi['<pad>']

model = Seq2Seq(enc, dec, SRC_PAD_IDX, TRG_PAD_IDX, device).to(device)

In [ ]:
# Load model
trained_model = os.path.join(BASE_DIR,"model/tut6-model_20210311_run4_glove_moredata.pt")
model.load_state_dict(torch.load(trained_model));
model.eval()

Seq2Seq(
  (encoder): Encoder(
    (tok_embedding): Embedding(2103, 256)
    (pos_embedding): Embedding(1000, 256)
    (layers): ModuleList(
      (0): EncoderLayer(
        (self_attn_layer_norm): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
        (ff_layer_norm): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
        (self_attention): MultiHeadAttentionLayer(
          (fc_q): Linear(in_features=256, out_features=256, bias=True)
          (fc_k): Linear(in_features=256, out_features=256, bias=True)
          (fc_v): Linear(in_features=256, out_features=256, bias=True)
          (fc_o): Linear(in_features=256, out_features=256, bias=True)
          (dropout): Dropout(p=0.15, inplace=False)
        )
        (positionwise_feedforward): PositionwiseFeedforwardLayer(
          (fc_1): Linear(in_features=256, out_features=512, bias=True)
          (fc_2): Linear(in_features=512, out_features=256, bias=True)
          (dropout): Dropout(p=0.15, inplace=False)
        )
  

In [ ]:
def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    s = re.sub(r"\s+", r" ", s).strip()
    return s


def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

In [ ]:
def evaluateInput(src_field,trg_field,trg_itos,model, device):
    input_sentence = ''
    while(1):
        try:
            # Get input sentence
            print('\n')
            input_sentence = input('Enter the Description > ')
            # Check if it is quit case
            if input_sentence == 'q' or input_sentence == 'quit': break
            # Normalize sentence
            input_sentence = normalizeString(input_sentence)
            # Evaluate sentence
            output_words,_ = helper.translate_sentence(input_sentence,src_field,trg_field,trg_itos, model, device)
            # Format and print response sentence
            output_words[:] = [x for x in output_words if not (x == '<eos>' or x == '<pad>')]
            print('Bot:\n', ' '.join(output_words))

        except KeyError:
            print("Error: Encountered unknown word.")

In [ ]:
evaluateInput(src_stoi,trg_stoi,trg_itos,model, device)



Enter the Description > write a program to add two numbers
Bot:
  num1 = 1.5 
  num2 = 6.3 
  sum = num1 + num2 
  print ( f'Sum: {sum}' ) 


Enter the Description > write a function to multiply two numbers
Bot:
  def gcd ( num1 , num2 ) : 
 	 return num1 * num2 


Enter the Description > write a function to find fibonacci number
Bot:
  def fib ( n ) : 
 	 if n == 1 : 
 		 return 0 
 	 elif n == 1 : 
 		 return 1 
 	 else : 
 		 return Fibonacci ( n - 1 ) + Fibonacci ( n - 2 ) 


Enter the Description > write a program to replace a string
Bot:
  
  str1 = 'CoffeeDay is best for coffee and having long conversations' 
  word_list = [ 'a' , 'long' ] 
  repl_word = 'good' 
  print ( "The original string is : " + str ( test_str ) ) 


Enter the Description > write a program to find log of a number
Bot:
  import math 
  def square ( num ) : 
 	 return math . exp ( x ) 


Enter the Description > write a function to find area of a circle
Bot:
  def findArea ( r ) : 
 	 PI = 3.142 
 	 return 